# Tutorial: Compressor Unit Model with IAPWS Property Package



![](compressor_2.svg)

## Learning Outcomes

- Demonstrate use of the compressor unit model in IDAES
- Demonstrate different simulation options available

In this tutorial, we will simulate sCO2 flow through a compressor unit model, using the Span-Wager CO2 property package.
It is assumed that the compressor operates at steady state.
The inlet specifications are as follows:

* Flow Rate = 1 kmol/hr
* Mole fraction (H2O) = 1
* Pressure = 101325 Pa
* Temperature = 390 K 

We will simulate 2 different cases, depending on the operating specifications by the user:

**Case 1**: The compressor specifications are as follows:
   * Pressure Increase = 50000 Pa
   * Isentropic Efficiency = 0.9
   
**Case 2**: The compressor specifications are as follows:
   * Pressure Ratio = 1.49346
   
Compressor specification:
   * Isentropic Efficiency = 0.9

IDAES documentation reference for compressor model:https://idaes-pse.readthedocs.io/en/stable/